In [1]:
import pandas as pd 
import os 
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()
caminho_arquivo = os.getenv("CSV_PATH_01")
df = pd.read_csv(caminho_arquivo, delimiter=';', encoding="latin1", low_memory=False)
df.head(5)

,ï»¿Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,CE,SOBRAL,ECONOGÃS DO BRASIL DIST. DERIV. DE PET. BIOC....,08.775.979/0002-62,RUA TABELIÃO IDELFONSO CAVALCANTI,455,NaN,CENTRO,62010-000,GASOLINA,01/01/2025,"6,29",NaN,R$ / litro,RAIZEN
1,NE,CE,SOBRAL,ECONOGÃS DO BRASIL DIST. DERIV. DE PET. BIOC....,08.775.979/0002-62,RUA TABELIÃO IDELFONSO CAVALCANTI,455,NaN,CENTRO,62010-000,GASOLINA ADITIVADA,01/01/2025,"6,49",NaN,R$ / litro,RAIZEN
2,NE,CE,SOBRAL,ECONOGÃS DO BRASIL DIST. DERIV. DE PET. BIOC....,08.775.979/0002-62,RUA TABELIÃO IDELFONSO CAVALCANTI,455,NaN,CENTRO,62010-000,DIESEL S10,01/01/2025,"6,19",NaN,R$ / litro,RAIZEN
3,NE,CE,SOBRAL,ECONOGÃS DO BRASIL DIST. DERIV. DE PET. BIOC....,08.775.979/0002-62,RUA TABELIÃO IDELFONSO CAVALCANTI,455,NaN,CENTRO,62010-000,ETANOL,01/01/2025,"5,19",NaN,R$ / litro,RAIZEN
4,NE,CE,SOBRAL,V.C.EMPREENDIMENTOS LTDA,03.551.935/0002-35,AVENIDA JOSE EUCLIDES FERREIRA GOMES,30,POSTO FLASH,CORACAO DE JESUS,62043-070,GASOLINA,01/01/2025,"6,53",NaN,R$ / litro,RAIZEN


In [2]:
#DATAFRAME ESTADOS
df_estados = df[['ï»¿Regiao - Sigla', 'Estado - Sigla']].drop_duplicates()
colunas_estados_tratadas = {
    'ï»¿Regiao - Sigla' : 'Regiao_sigla',
    'Estado - Sigla' : 'Estado_sigla'
}
df_estados.rename(columns=colunas_estados_tratadas, inplace=True)
df_estados['Estado_id'] = df_estados.index +1
df_estados = df_estados[['Estado_id', 'Regiao_sigla', 'Estado_sigla']]
df_estados.head(5)

,Estado_id,Regiao_sigla,Estado_sigla
0,1,NE,CE
24,25,CO,DF
28,29,SE,MG
54,55,SE,RJ
101,102,NE,RN


In [3]:
#DATAFRAME MUNICIPIO
df_municipio = df[['Municipio', 'Estado - Sigla']].drop_duplicates()
df_municipio.rename(columns={'Estado - Sigla' : 'Estado_sigla'}, inplace=True)
df_municipio['Municipio'] = df_municipio['Municipio'].str.title()
df_municipio['Municipio_id'] = df_municipio.index +1
df_municipio = df_municipio.merge(df_estados[['Estado_id', 'Estado_sigla' ]], on= 'Estado_sigla')
df_municipio = df_municipio[['Municipio_id', 'Estado_id', 'Municipio', 'Estado_sigla']]
df_municipio.head(5)

,Municipio_id,Estado_id,Municipio,Estado_sigla
0,1,1,Sobral,CE
1,25,25,Brasilia,DF
2,29,29,Uberlandia,MG
3,55,55,Campos Dos Goytacazes,RJ
4,102,102,Caico,RN


In [4]:
#DATAFRAME REVENDAS
df_revendas = df[['Revenda', 'CNPJ da Revenda', 'Bandeira', 'Municipio']].drop_duplicates()
df_revendas.rename(columns={"CNPJ da Revenda" : "CNPJ"}, inplace=True)
df_revendas['Revenda'] = df_revendas['Revenda'].str.title()
df_revendas['CNPJ'] = df_revendas['CNPJ'].astype(str)
df_revendas['Bandeira'] = df_revendas['Bandeira'].str.title()
df_revendas['Municipio'] = df_revendas['Municipio'].str.title()
df_revendas['Revenda_id'] = df_revendas.index +1
df_revendas = df_revendas.merge(df_municipio[['Municipio_id', 'Municipio']], on='Municipio')
df_revendas = df_revendas[['Revenda_id', 'Municipio_id', 'Revenda', 'CNPJ', 'Bandeira', 'Municipio']]
df_revendas.head(5)

,Revenda_id,Municipio_id,Revenda,CNPJ,Bandeira,Municipio
0,1,1,EconogãS Do Brasil Dist. Deriv. De Pet. Bioc....,08.775.979/0002-62,Raizen,Sobral
1,5,1,V.C.Empreendimentos Ltda,03.551.935/0002-35,Raizen,Sobral
2,9,1,Center Posto Comercial Ltda,02.905.892/0001-03,Vibra,Sobral
3,13,1,V.C.Empreendimentos Ltda,03.551.935/0001-54,Ipiranga,Sobral
4,17,1,Posto Sao Domingos Ltda,69.366.128/0001-70,Raizen,Sobral


In [5]:
#DATAFRAME PREÇOS
df_precos = df[['Produto', 'Data da Coleta', 'Valor de Venda', 'Valor de Compra', 
                'Unidade de Medida', 'Municipio', 'Revenda' ]].drop_duplicates()
colunas_precos_tratadas = {
    'Data da Coleta' : 'Data_coleta',
    'Valor de Venda' : 'Valor_venda',
    'Valor de Compra' : 'Valor_compra',
    'Unidade de Medida' : 'Unidade_medida'
}
df_precos.rename(columns=colunas_precos_tratadas, inplace=True)
df_precos['Data_coleta'] = pd.to_datetime(df_precos['Data_coleta'], dayfirst=True).dt.strftime("%Y-%m-%d")
df_precos['Preco_id'] = df_precos.index +1
df_precos['Valor_compra'] = df_precos['Valor_compra'].fillna('Valor não informado')
df_precos['Produto'] = df_precos['Produto'].str.title()
df_precos['Municipio'] = df_precos['Municipio'].str.title()
df_precos['Revenda'] = df_precos['Revenda'].str.title()
df_precos = df_precos.merge(df_municipio[['Municipio_id', 'Municipio']], on = 'Municipio')
df_precos = df_precos.merge(df_revendas[['Revenda_id', 'Revenda']], on = 'Revenda')
df_precos = df_precos[['Preco_id', 'Revenda_id', 'Produto', 'Municipio', 'Revenda', 
                         'Data_coleta', 'Valor_venda', 'Valor_compra', 'Unidade_medida']]
df_precos.head(5)


,Preco_id,Revenda_id,Produto,Municipio,Revenda,Data_coleta,Valor_venda,Valor_compra,Unidade_medida
0,1,1,Gasolina,Sobral,EconogãS Do Brasil Dist. Deriv. De Pet. Bioc....,2025-01-01,"6,29",Valor não informado,R$ / litro
1,2,1,Gasolina Aditivada,Sobral,EconogãS Do Brasil Dist. Deriv. De Pet. Bioc....,2025-01-01,"6,49",Valor não informado,R$ / litro
2,3,1,Diesel S10,Sobral,EconogãS Do Brasil Dist. Deriv. De Pet. Bioc....,2025-01-01,"6,19",Valor não informado,R$ / litro
3,4,1,Etanol,Sobral,EconogãS Do Brasil Dist. Deriv. De Pet. Bioc....,2025-01-01,"5,19",Valor não informado,R$ / litro
4,5,5,Gasolina,Sobral,V.C.Empreendimentos Ltda,2025-01-01,"6,53",Valor não informado,R$ / litro


In [6]:
#DATAFRAME ENDEREÇO 
df_endereco = df[['Nome da Rua', 'Numero Rua', 'Complemento', 'Bairro', 'Cep', 'Municipio']].drop_duplicates()
colunas_endereco_tratadas = {
    'Nome da Rua' : 'Nome_rua',
    'Numero Rua' : 'Numero_rua',
    'Cep' : 'CEP'
}
df_endereco.rename(columns=colunas_endereco_tratadas, inplace=True)
df_endereco['Endereco_id'] = df_endereco.index +1
df_endereco['Nome_rua'] = df_endereco['Nome_rua'].str.title()
df_endereco['Complemento'] = df_endereco['Complemento'].fillna("Complemento não informado")
df_endereco['Complemento'] = df_endereco['Complemento'].str.title()
df_endereco['CEP'] = df_endereco['CEP'].astype(str)
df_endereco['Numero_rua'] = df_endereco['Numero_rua'].astype(str)
df_endereco['Numero_rua'] = df_endereco['Numero_rua'].fillna("Numero não informado")
df_endereco['Bairro'] = df_endereco['Bairro'].str.title()
df_endereco['Municipio'] = df_endereco['Municipio'].str.title()
df_endereco = df_endereco.merge(df_municipio[['Municipio_id', 'Municipio']], on= 'Municipio')
df_endereco = df_endereco[['Endereco_id', 'Municipio_id', 'Nome_rua', 'Numero_rua',
                            'Complemento', 'Bairro', 'CEP', 'Municipio']]
df_endereco.head(5)

,Endereco_id,Municipio_id,Nome_rua,Numero_rua,Complemento,Bairro,CEP,Municipio
0,1,1,Rua TabeliãO Idelfonso Cavalcanti,455,Complemento Não Informado,Centro,62010-000,Sobral
1,5,1,Avenida Jose Euclides Ferreira Gomes,30,Posto Flash,Coracao De Jesus,62043-070,Sobral
2,9,1,Avenida Jose Arimateia Monte E Silva,425,Complemento Não Informado,Campos Dos Velhos,62114-000,Sobral
3,13,1,Avenida Monsenhor Jose Aloisio Pinto,80,Loja 01,Dom Expedito,62050-255,Sobral
4,17,1,Avenida Doutor Guarani,854,Complemento Não Informado,Jocely Dantas De Andrade Torres,62042-030,Sobral


In [7]:
#DATAFRAME ENDEREÇO 
df_endereco = df[['Nome da Rua', 'Numero Rua', 'Complemento', 'Bairro', 'Cep', 'Municipio']].drop_duplicates()
colunas_endereco_tratadas = {
    'Nome da Rua' : 'Nome_rua',
    'Numero Rua' : 'Numero_rua',
    'Cep' : 'CEP'
}
df_endereco.rename(columns=colunas_endereco_tratadas, inplace=True)
df_endereco['Endereco_id'] = df_endereco.index +1
df_endereco['Nome_rua'] = df_endereco['Nome_rua'].str.title()
df_endereco['Complemento'] = df_endereco['Complemento'].fillna("Complemento não informado")
df_endereco['Complemento'] = df_endereco['Complemento'].str.title()
df_endereco['CEP'] = df_endereco['CEP'].astype(str)
df_endereco['Numero_rua'] = df_endereco['Numero_rua'].astype(str)
df_endereco['Numero_rua'] = df_endereco['Numero_rua'].fillna("Numero não informado")
df_endereco['Bairro'] = df_endereco['Bairro'].str.title()
df_endereco['Municipio'] = df_endereco['Municipio'].str.title()
df_endereco = df_endereco.merge(df_municipio[['Municipio_id', 'Municipio']], on= 'Municipio')
df_endereco = df_endereco[['Endereco_id', 'Municipio_id', 'Nome_rua', 'Numero_rua',
                            'Complemento', 'Bairro', 'CEP', 'Municipio']]
df_endereco.head()

,Endereco_id,Municipio_id,Nome_rua,Numero_rua,Complemento,Bairro,CEP,Municipio
0,1,1,Rua TabeliãO Idelfonso Cavalcanti,455,Complemento Não Informado,Centro,62010-000,Sobral
1,5,1,Avenida Jose Euclides Ferreira Gomes,30,Posto Flash,Coracao De Jesus,62043-070,Sobral
2,9,1,Avenida Jose Arimateia Monte E Silva,425,Complemento Não Informado,Campos Dos Velhos,62114-000,Sobral
3,13,1,Avenida Monsenhor Jose Aloisio Pinto,80,Loja 01,Dom Expedito,62050-255,Sobral
4,17,1,Avenida Doutor Guarani,854,Complemento Não Informado,Jocely Dantas De Andrade Torres,62042-030,Sobral
